In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd

import cartopy.crs as ccrs
import cartopy.feature as cf
import scipy.stats as stats

from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

from shapely.geometry import Point, MultiPolygon

import warnings
warnings.filterwarnings('ignore')

## Complete date in CPTEC clear rain gauges

In [2]:
year = 2024

In [3]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Brazil','OBS_CPTEC')

In [4]:
file_list = glob.glob(os.path.join(dir_base,'1hr',str(year),'clear','*.csv'))
file_list = sorted(file_list)
file_len = len(file_list)
print(f'Number of Stations: {file_len}')

Number of files: 3442


In [7]:
TIME_REF = pd.date_range(
            start=f'{year}-01-01 0:00:00', 
            end=f'{year}-12-31 23:00:00', 
            freq='1h').strftime("%Y-%m-%d %H:%M:%S")
TIME_PD = pd.DataFrame({'date':TIME_REF})
print(f'Number of times: {len(TIME_PD)}')

Number of times: 8784


In [6]:
for nn in range(file_len):#file_len
    data = pd.read_csv(file_list[nn])

    outname = os.path.join(dir_base, '1hr', str(year), 'rdy', file_list[nn].split('/')[-1])

    if os.path.exists(outname):
        continue

    else:
        time_vecto = data.date.values
        time_vecto = pd.to_datetime(time_vecto)

        if len(data) < len(TIME_PD):
            print('WARNING: RAINGAUGE time is not complete. Fill times with NaN')
            OBS_FILL = pd.merge(data, TIME_PD, on='date', how='right')
        elif len(data) == len(TIME_PD):
            continue
        else:
            print('ERROR: RAINGAUGE has too much times, verify!!')

        cod_value = OBS_FILL['code'].dropna().unique()[0]
        lon_value = OBS_FILL['lon'].dropna().unique()[0]
        lat_value = OBS_FILL['lat'].dropna().unique()[0]
        alt_value = OBS_FILL['alt'].dropna().unique()[0]

        OBS_FILL['code'] = OBS_FILL['code'].fillna(cod_value)
        OBS_FILL['lon'] = OBS_FILL['lon'].fillna(lon_value)
        OBS_FILL['lat'] = OBS_FILL['lat'].fillna(lat_value)
        OBS_FILL['alt'] = OBS_FILL['alt'].fillna(alt_value)

        OBS_FILL.to_csv(outname, header=True, index=False)